# Creating voting affinity scores for members of the 118th congress

In [26]:
import numpy as np
import pandas as pd

In [27]:
votes = pd.read_csv('https://voteview.com/static/data/out/votes/HS118_votes.csv')

In [28]:
votes = votes[['chamber', 'rollnumber', 'icpsr', 'cast_code']]
#house_votes = votes.query("chamber == 'House'")
senate_votes = votes.query("chamber == 'Senate'") 

In [29]:
paired_votes = pd.merge(senate_votes, senate_votes, on = ['chamber', 'rollnumber'], how='inner', validate = 'many_to_many')

In [30]:
paired_votes = paired_votes.rename({'icpsr_x': 'icpsr', 'icpsr_y': 'member_compare',
                                   'cast_code_x': 'vote',
                                   'cast_code_y': 'compare_vote'}, axis=1)



In [37]:
paired_votes = paired_votes.query("icpsr != member_compare")
paired_votes

,chamber,rollnumber,icpsr,vote,member_compare,compare_vote,equal_vote
1,Senate,1,14226,1,14435,1,True
2,Senate,1,14226,1,14858,1,True
3,Senate,1,14226,1,14871,1,True
4,Senate,1,14226,1,14921,1,True
5,Senate,1,14226,1,15015,1,True
...,...,...,...,...,...,...,...
2529198,Senate,253,91300,1,42303,6,False
2529199,Senate,253,91300,1,42304,6,False
2529200,Senate,253,91300,1,42305,1,True
2529201,Senate,253,91300,1,49308,1,True


In [38]:
paired_votes = paired_votes.assign(
    equal_vote = paired_votes['vote'] == paired_votes['compare_vote']
)

In [39]:
paired_votes

,chamber,rollnumber,icpsr,vote,member_compare,compare_vote,equal_vote
1,Senate,1,14226,1,14435,1,True
2,Senate,1,14226,1,14858,1,True
3,Senate,1,14226,1,14871,1,True
4,Senate,1,14226,1,14921,1,True
5,Senate,1,14226,1,15015,1,True
...,...,...,...,...,...,...,...
2529198,Senate,253,91300,1,42303,6,False
2529199,Senate,253,91300,1,42304,6,False
2529200,Senate,253,91300,1,42305,1,True
2529201,Senate,253,91300,1,49308,1,True


In [41]:
paired_votes = paired_votes[['icpsr', 'member_compare', 'equal_vote']]
paired_votes

,icpsr,member_compare,equal_vote
1,14226,14435,True
2,14226,14858,True
3,14226,14871,True
4,14226,14921,True
5,14226,15015,True
...,...,...,...
2529198,91300,42303,False
2529199,91300,42304,False
2529200,91300,42305,True
2529201,91300,49308,True


In [44]:
affinity = paired_votes.groupby(['icpsr', 'member_compare']).agg({'equal_vote': 'mean'})\
.reset_index()

In [52]:
affinity['affinity_score'] = round(100*affinity['equal_vote'], 1).astype('str') + '%'

In [53]:
affinity

,icpsr,member_compare,equal_vote,affinity_score
0,14226,14435,0.308300,30.8%
1,14226,14858,0.379447,37.9%
2,14226,14871,0.355731,35.6%
3,14226,14921,0.766798,76.7%
4,14226,15015,0.359684,36.0%
...,...,...,...,...
10093,91300,42304,0.177866,17.8%
10094,91300,42305,1.000000,100.0%
10095,91300,49300,0.485597,48.6%
10096,91300,49308,0.913043,91.3%


In [54]:
affinity = affinity.drop('equal_vote', axis=1)

In [55]:
affinity

,icpsr,member_compare,affinity_score
0,14226,14435,30.8%
1,14226,14858,37.9%
2,14226,14871,35.6%
3,14226,14921,76.7%
4,14226,15015,36.0%
...,...,...,...
10093,91300,42304,17.8%
10094,91300,42305,100.0%
10095,91300,49300,48.6%
10096,91300,49308,91.3%


In [56]:
ideology = pd.read_csv('https://voteview.com/static/data/out/members/HS118_members.csv')

In [58]:
ideo_senate = ideology.query('chamber =="Senate"')

In [60]:
ideo_senate = ideo_senate[['icpsr', 'state_abbrev', 'party_code', 'bioname']]
ideo_senate

,icpsr,state_abbrev,party_code,bioname
441,42102,AL,200,"TUBERVILLE, Thomas Hawley (Tommy)"
442,42300,AL,200,"BRITT, Katie Elizabeth"
443,40300,AK,200,"MURKOWSKI, Lisa"
444,41500,AK,200,"SULLIVAN, Daniel Scott"
445,41905,AZ,100,"KELLY, Mark Edward"
...,...,...,...,...
537,40915,WV,100,"MANCHIN, Joe, III"
538,29940,WI,100,"BALDWIN, Tammy"
539,41111,WI,200,"JOHNSON, Ron"
540,20953,WY,200,"LUMMIS, Cynthia M."


In [61]:
affinity = pd.merge(affinity, ideo_senate, on = 'icpsr', how = 'outer', validate = 'many_to_one')

In [62]:
affinity

,icpsr,member_compare,affinity_score,state_abbrev,party_code,bioname
0,14226,14435,30.8%,IA,200,"GRASSLEY, Charles Ernest"
1,14226,14858,37.9%,IA,200,"GRASSLEY, Charles Ernest"
2,14226,14871,35.6%,IA,200,"GRASSLEY, Charles Ernest"
3,14226,14921,76.7%,IA,200,"GRASSLEY, Charles Ernest"
4,14226,15015,36.0%,IA,200,"GRASSLEY, Charles Ernest"
...,...,...,...,...,...,...
10093,91300,42304,17.8%,AZ,328,"SINEMA, Kyrsten"
10094,91300,42305,100.0%,AZ,328,"SINEMA, Kyrsten"
10095,91300,49300,48.6%,AZ,328,"SINEMA, Kyrsten"
10096,91300,49308,91.3%,AZ,328,"SINEMA, Kyrsten"


In [63]:
affinity = pd.merge(affinity, ideo_senate, left_on = 'member_compare', right_on = 'icpsr', how = 'outer', validate = 'many_to_one')

In [64]:
affinity

,icpsr_x,member_compare,affinity_score,state_abbrev_x,party_code_x,bioname_x,icpsr_y,state_abbrev_y,party_code_y,bioname_y
0,14226,14435,30.8%,IA,200,"GRASSLEY, Charles Ernest",14435,MA,100,"MARKEY, Edward John"
1,14858,14435,90.9%,NY,100,"SCHUMER, Charles Ellis (Chuck)",14435,MA,100,"MARKEY, Edward John"
2,14871,14435,93.3%,OR,100,"WYDEN, Ronald Lee",14435,MA,100,"MARKEY, Edward John"
3,14921,14435,24.1%,KY,200,"McCONNELL, Addison Mitchell (Mitch)",14435,MA,100,"MARKEY, Edward John"
4,15015,14435,92.1%,DE,100,"CARPER, Thomas Richard",14435,MA,100,"MARKEY, Edward John"
...,...,...,...,...,...,...,...,...,...,...
10093,42305,14226,33.3%,CA,100,"BUTLER, Laphonza Romanique",14226,IA,200,"GRASSLEY, Charles Ernest"
10094,49300,14226,14.0%,CA,100,"FEINSTEIN, Dianne",14226,IA,200,"GRASSLEY, Charles Ernest"
10095,49308,14226,36.4%,WA,100,"MURRAY, Patty",14226,IA,200,"GRASSLEY, Charles Ernest"
10096,49703,14226,60.1%,ME,200,"COLLINS, Susan Margaret",14226,IA,200,"GRASSLEY, Charles Ernest"


In [68]:
affinity.query('state_abbrev_x == "VA"').sort_values('affinity_score')

,icpsr_x,member_compare,affinity_score,state_abbrev_x,party_code_x,bioname_x,icpsr_y,state_abbrev_y,party_code_y,bioname_y
9569,41305,42305,100.0%,VA,100,"KAINE, Timothy Michael (Tim)",42305,CA,100,"BUTLER, Laphonza Romanique"
9554,40909,42305,100.0%,VA,100,"WARNER, Mark",42305,CA,100,"BUTLER, Laphonza Romanique"
8269,41305,41903,11.9%,VA,100,"KAINE, Timothy Michael (Tim)",41903,FL,200,"SCOTT, Richard Lynn (Rick)"
8669,41305,42102,11.9%,VA,100,"KAINE, Timothy Michael (Tim)",42102,AL,200,"TUBERVILLE, Thomas Hawley (Tommy)"
8254,40909,41903,12.3%,VA,100,"WARNER, Mark",41903,FL,200,"SCOTT, Richard Lynn (Rick)"
...,...,...,...,...,...,...,...,...,...,...
4568,41305,40700,98.4%,VA,100,"KAINE, Timothy Michael (Tim)",40700,MN,100,"KLOBUCHAR, Amy"
7569,41305,41702,98.4%,VA,100,"KAINE, Timothy Michael (Tim)",41702,NH,100,"HASSAN, Margaret (Maggie)"
5768,41305,41101,98.8%,VA,100,"KAINE, Timothy Michael (Tim)",41101,CT,100,"BLUMENTHAL, Richard"
4268,41305,39310,99.2%,VA,100,"KAINE, Timothy Michael (Tim)",39310,WA,100,"CANTWELL, Maria E."
